In [182]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime
from urllib.request import urlopen
import quandl
from pandas_datareader.quandl import QuandlReader

In [3]:
quandl.ApiConfig.api_key = 'EZp1zRismtTHsmGpjd2S'

In [174]:
# Point 1
def filter_data(tickers=[], start_date='', end_date='', additional_columns=[]):
    data = pd.read_csv("CoreUSFund/SHARADAR-SF1.csv")

    data = data.loc[data['ticker'].isin(tickers)]
    data = data[data['calendardate'].between(start_date, end_date, inclusive=True)]
    data = data.filter(items=['ticker', 'calendardate'] + additional_columns)
    
    return data

In [176]:
# filter_data(['ZZ', 'ZYXI'], '2007-12-30', '2010-12-31', ['accoci', 'fcf'])

In [179]:
# Point 2
def filter_non_num_data(column_names, column_values):
    data = pd.read_csv("TickersAndMetaData/SHARADAR-TICKERS.csv")
    
    for column_name in column_names:
        data = data.loc[data[column_name].isin(column_values)]
        
    return data['ticker'].to_numpy().tolist()


def filter_num_data(tickers=[], start_date='', end_date='', additional_columns=[]):
    data = pd.read_csv("CoreUSFund/SHARADAR-SF1.csv")
    
    if len(tickers) > 0:
        data = data.loc[data['ticker'].isin(tickers)]
    
    if start_date != '' and end_date != '':
        data = data[data['calendardate'].between(start_date, end_date, inclusive=True)]
    elif start_date != '' and end_date == '':
        data = data[data['calendardate'].between(start_date, datetime.date.today().strftime('%Y-%m-%d'), inclusive=True)]
    elif start_date == '' and end_date != '':
        data = data[data['calendardate'].between('1998-01-01', end_date, inclusive=True)]
        
    column_names = []
    for additional_column_object in additional_columns:
        column_names.append(additional_column_object['column_name'])
        
    data = data.filter(items=['ticker', 'calendardate'] + column_names)
    
    for additional_column_object in additional_columns:
        for column_name in column_names:
            if additional_column_object['artefact'] == '>':
                data = data[data[column_name] > additional_column_object['extreme_value']]
            elif additional_column_object['artefact'] == '<':
                data = data[data[column_name] < additional_column_object['extreme_value']]
            elif additional_column_object['artefact'] == '=':
                data = data[data[column_name] == additional_column_object['extreme_value']]
            elif additional_column_object['artefact'] == '>=':
                data = data[data[column_name] >= additional_column_object['extreme_value']]
            elif additional_column_object['artefact'] == '<=':
                data = data[data[column_name] <= additional_column_object['extreme_value']]
                
    return data

In [180]:
# filter_num_data(additional_columns=[{'column_name': 'assetturnover', 'extreme_value': 1.6, 'artefact': '<='}])

In [269]:
# Point 3
def filter_and_sort_data(tickers=[], start_date='', end_date='', sort_by=[]):
    final_df = pd.DataFrame(columns=['ticker', 'momentum_score', 'pe_ratio', 'dividend_yield'])
    data = pd.read_csv("EquityPrices/SHARADAR-SEP.csv")
    data_sf1 = pd.read_csv("CoreUSFund/SHARADAR-SF1.csv")
    
    # Momentum score, p/e ratio, dividend yield
    data = data[data['date'].between(start_date, end_date, inclusive=True)]
    data_sf1 = data_sf1[data_sf1['calendardate'].between(start_date, end_date, inclusive=True)]
    
    if len(tickers) > 0:
        data = data.loc[data['ticker'].isin(tickers)]
        data_sf1 = data_sf1.loc[data_sf1['ticker'].isin(tickers)]
        
    if start_date != '' and end_date != '':
        data = data[data['date'].between(start_date, end_date, inclusive=True)]
        data_sf1 = data_sf1[data_sf1['calendardate'].between(start_date, end_date, inclusive=True)]
    elif start_date != '' and end_date == '':
        data = data[data['date'].between(start_date, datetime.date.today().strftime('%Y-%m-%d'), inclusive=True)]
        data_sf1 = data_sf1[data_sf1['calendardate'].between(start_date, datetime.date.today().strftime('%Y-%m-%d'), inclusive=True)]
    elif start_date == '' and end_date != '':
        data = data[data['date'].between('1998-01-01', end_date, inclusive=True)]
        data_sf1 = data_sf1[data_sf1['calendardate'].between('1998-01-01', end_date, inclusive=True)]
        
    unique_tickers = data['ticker'].unique()
    helper_data = pd.DataFrame()
    helper_data_sf1 = pd.DataFrame()
    momentum_score = 0
    pe_ratio = 0
    dividend_yield = 0
    for ticker in unique_tickers:
        helper_data = data.loc[data['ticker'].isin([ticker])]
        helper_data_sf1 = data_sf1.loc[data_sf1['ticker'].isin([ticker])]
        
        momentum_score = helper_data['close'].iloc[0] / helper_data['close'].iloc[-1] - 1
        pe_ratio = helper_data_sf1['pe'].iloc[0]
        dividend_yield = helper_data_sf1['divyield'].iloc[0]
        
        
        final_df = final_df.append(pd.DataFrame(columns=['ticker', 'momentum_score', 'pe_ratio', 'dividend_yield'],
                                                data=[[ticker, momentum_score, pe_ratio, dividend_yield]]))
    
    if (len(sort_by) == 0):
        return final_df
    
    return final_df.sort_values(by=sort_by)

In [272]:
# filter_and_sort_data([], '2003-01-01', '2010-01-01')

In [322]:
# Point 4
# a) Inverse Volatility
def inverse_volatility(tickers=[], start_date='', end_date=''):
    data = pd.read_csv("EquityPrices/SHARADAR-SEP.csv")
    final_df = pd.DataFrame(columns=['ticker', 'portfolio_weight'])
    
    if len(tickers) > 0:
        data = data.loc[data['ticker'].isin(tickers)]
        
    unique_tickers = data['ticker'].unique()
    
    if start_date != '' and end_date != '':
        data = data[data['date'].between(start_date, end_date, inclusive=True)]
    elif start_date != '' and end_date == '':
        data = data[data['date'].between(start_date, datetime.date.today().strftime('%Y-%m-%d'), inclusive=True)]
    elif start_date == '' and end_date != '':
        data = data[data['date'].between('1998-01-01', end_date, inclusive=True)]
    
    for ticker in unique_tickers:
        helper_arr = []
        helper_data = data.loc[data['ticker'].isin([ticker])]
        for i in range(len(helper_data['ticker'])):
            if i == 0:
                continue
            helper_arr.append(helper_data['close'].iloc[i] / helper_data['close'].iloc[i-1] - 1)
        
        inverse_volatility = 1 / (np.array(helper_arr).std() * np.sqrt(252))
        
        final_df = final_df.append(pd.DataFrame(columns=['ticker', 'portfolio_weight'],
                                                data=[[ticker, inverse_volatility]]))
    
    final_df.loc[:, 'portfolio_weight'] = final_df['portfolio_weight'].apply(lambda x: x / np.sum(final_df['portfolio_weight']))
    
    return final_df

In [324]:
# inverse_volatility(['ZZ', 'ZYTO'], '2009-01-01', '2010-01-01')

In [347]:
# Point 5
def percentiles(data, segment_rank, number_of_segments):
    data = data.iloc[int((segment_rank - 1) * len(data['ticker']) / number_of_segments) : 
                     int(segment_rank * len(data['ticker']) / number_of_segments)]
    
    return data

In [348]:
# test_data = filter_and_sort_data([], '2003-01-01', '2010-01-01')

In [354]:
# percentiles(test_data, 2, 3)

,ticker,momentum_score,pe_ratio,dividend_yield
0,ZYXI,-0.624113,13.523,0.0
